# Results of audit on Maji Ndogo water project

In [1]:
import json
import pandas as pd
import mysql.connector
from tabulate import tabulate

# Step 1: Read the CSV file into a DataFrame using ';' as the separator
file_path = "C:/Users/craig/Desktop/Portfolio Series/Auditor_report.csv"  # Update with your file path
data = pd.read_csv(file_path, sep=';')

# Load database credentials from config.json
def load_config():
    with open("config.json", "r") as file:
        return json.load(file)

# Load the configuration
config = load_config()

# Function to create a connection to the MySQL database
def create_connection():
    connection = mysql.connector.connect(
        host=config["DB_HOST"],      
        user=config["DB_USER"],      
        password=config["DB_PASSWORD"],  
        database=config["DB_NAME"]      
    )
    return connection


# Step 2: Create the auditor_report table
try:
    connection = create_connection()  # Create the database connection
    cursor = connection.cursor()
    drop_table_query = "DROP TABLE IF EXISTS `auditor_report`"
    cursor.execute(drop_table_query)
    connection.commit()  # Commit the drop table operation

    create_table_query = """
    CREATE TABLE `auditor_report` (
        `location_id` VARCHAR(32),
        `type_of_water_source` VARCHAR(64),
        `true_water_source_score` INT DEFAULT NULL,
        `statements` VARCHAR(255)
    );
    """
    cursor.execute(create_table_query)
    connection.commit()  # Commit the create table operation
finally:
    cursor.close()  # Ensure the cursor is closed in the end

# Step 3: Insert data into the auditor_report table
try:
    connection = create_connection()  # Create a new database connection for insertion
    cursor = connection.cursor()  # Create a new cursor for insertion
    insert_query = """
    INSERT INTO auditor_report (location_id, type_of_water_source, true_water_source_score, statements)
    VALUES (%s, %s, %s, %s);
    """
    
    # Use executemany to insert all rows at once for better performance
    cursor.executemany(insert_query, data.values.tolist())

    connection.commit()  # Commit the transaction after all inserts
except SyntaxError as e:
    print(f"SyntaxError: {e}")
except Exception as e:
    print(f"Error: {e}")
finally:
    cursor.close()  # Ensure the cursor is closed in the end

In [2]:
# Function to execute a SELECT query and return the result as a DataFrame
def execute_query(query):
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute(query)
    
    # If it's a SELECT query, return a DataFrame
    if query.strip().upper().startswith("SELECT"):
        results = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]
        df = pd.DataFrame(results, columns=column_names)
        
        # Close cursor and connection
        cursor.close()
        conn.close()
        return df
    
    # If it's an UPDATE query, commit the changes and close the connection
    else:
        conn.commit()
        cursor.close()
        conn.close()
        print(f"Query executed: {query}")

# Function to color headers with a custom style
def color_header():
    return [{'selector': 'thead th', 'props': [('background-color', 'lightblue'),
                                               ('color', 'black'),
                                               ('font-size', '12pt'),
                                               ('text-align', 'center')]}]

# Function to apply cell formatting for individual DataFrame cells
def highlight_cells(val):
    return 'background-color: #f4f4f4'  # Light grey for cell background

# Step 4: Query to select location_id and true_water_source_score from auditor_report
query = """
SELECT location_id, true_water_source_score FROM auditor_report LIMIT 10;
"""

# Step 5: Execute the query and get the DataFrame
result_df = execute_query(query)

# Step 6: Apply custom table styles and display the styled DataFrame
styled_result_df = (
    result_df.style
    .applymap(highlight_cells)  # Apply custom function to highlight cells
    .set_table_styles(color_header())  # Apply custom header style
)

# Display the DataFrame with styles
print("Location IDs and True Water Source Scores:")
display(styled_result_df)

Location IDs and True Water Source Scores:


,location_id,true_water_source_score
0,SoRu34980,1
1,AkRu08112,3
2,AkLu02044,0
3,AkHa00421,3
4,SoRu35221,0
5,HaAm16170,1
6,AkRu04812,3
7,AkRu08304,3
8,AkRu05107,2
9,AkRu05215,3


In [3]:
# Function to color headers with a custom style
def color_header():
    return [{'selector': 'thead th', 'props': [('background-color', 'lightblue'),
                                               ('color', 'black'),
                                               ('font-size', '12pt'),
                                               ('text-align', 'center')]}]

# Function to apply cell formatting for individual DataFrame cells
def highlight_cells(val):
    return 'background-color: #f4f4f4'  # Light grey for cell background

# Step 3: SQL Query to join water_quality and auditor_report tables
query = """
SELECT
auditor_report.location_id AS audit_location,
auditor_report.true_water_source_score,
visits.location_id AS visit_location,
visits.record_id
FROM
auditor_report
JOIN
visits
ON auditor_report.location_id = visits.location_id LIMIT 10;
"""

# Step 3: Execute the query and get the DataFrame
joined_df = execute_query(query)

# Step 4: Apply custom table styles and display the styled DataFrame
styled_joined_df = (
    joined_df.style
    .applymap(highlight_cells)  # Apply custom function to highlight cells
    .set_table_styles(color_header())  # Apply custom header style
)

# Display the DataFrame with styles
print("Joined Data (Subjective Quality Score, Record ID, Location ID):")
display(styled_joined_df)

Joined Data (Subjective Quality Score, Record ID, Location ID):


,audit_location,true_water_source_score,visit_location,record_id
0,SoRu34980,1,SoRu34980,5185
1,AkRu08112,3,AkRu08112,59367
2,AkLu02044,0,AkLu02044,37379
3,AkHa00421,3,AkHa00421,51627
4,SoRu35221,0,SoRu35221,28758
5,HaAm16170,1,HaAm16170,31048
6,AkRu04812,3,AkRu04812,1513
7,AkRu08304,3,AkRu08304,1218
8,AkRu05107,2,AkRu05107,8322
9,AkRu05215,3,AkRu05215,21160


In [4]:
# Step 2: Query to join water_quality with auditor_report
query = """
SELECT
    auditor_report.location_id AS location_id,
    visits.record_id AS record_id,
    auditor_report.true_water_source_score AS auditor_score,
    water_quality.subjective_quality_score AS surveyor_score
FROM
    auditor_report
JOIN
    visits ON auditor_report.location_id = visits.location_id
JOIN
    water_quality ON visits.record_id = water_quality.record_id
LIMIT 10;
"""

# Step 3: Execute the query and get the DataFrame
joined_df = execute_query(query)

# Step 4: Drop any duplicate columns (if necessary) and clean up the column names
joined_df = joined_df.drop(columns=['location_id'], errors='ignore')  # Drop location_id if present
joined_df.rename(columns={
    'auditor_score': 'auditor_score',  # Just to clarify
    'surveyor_score': 'surveyor_score'
}, inplace=True)

# Step 5: Apply custom table styles and display the styled DataFrame
styled_joined_df = (
    joined_df.style
    .applymap(highlight_cells)  # Apply custom function to highlight cells
    .set_table_styles(color_header())  # Apply custom header style
)

# Display the DataFrame with styles
print("Cleaned Data (Record ID, Auditor Score, Surveyor Score):")
display(styled_joined_df)

Cleaned Data (Record ID, Auditor Score, Surveyor Score):


,record_id,auditor_score,surveyor_score
0,5185,1,1
1,59367,3,3
2,37379,0,0
3,51627,3,3
4,28758,0,0
5,31048,1,1
6,1513,3,3
7,1218,3,3
8,8322,2,2
9,21160,3,10


In [5]:
# Step 2: Query to join water_quality with auditor_report
query = """
SELECT
    auditor_report.location_id AS location_id,
    visits.record_id AS record_id,
    auditor_report.true_water_source_score AS auditor_score,
    water_quality.subjective_quality_score AS surveyor_score
FROM
    auditor_report
JOIN
    visits ON auditor_report.location_id = visits.location_id
JOIN
    water_quality ON visits.record_id = water_quality.record_id
WHERE
    visits.visit_count = 1  -- Filter for unique visits
LIMIT 10;
"""

# Step 3: Execute the query and get the DataFrame
joined_df = execute_query(query)

# Step 4: Drop any duplicate columns (if necessary) and clean up the column names
joined_df = joined_df.drop(columns=['location_id'], errors='ignore')  # Drop location_id if present
joined_df.rename(columns={
    'auditor_score': 'auditor_score',  # Just to clarify
    'surveyor_score': 'surveyor_score'
}, inplace=True)

# Step 5: Apply custom table styles and display the styled DataFrame
styled_joined_df = (
    joined_df.style
    .applymap(highlight_cells)  # Apply custom function to highlight cells
    .set_table_styles(color_header())  # Apply custom header style
)

# Display the DataFrame with styles
print("Location ID, Record ID, Auditor Score, Surveyor Score (for unique visits):")
display(styled_joined_df)

Location ID, Record ID, Auditor Score, Surveyor Score (for unique visits):


,record_id,auditor_score,surveyor_score
0,5185,1,1
1,59367,3,3
2,37379,0,0
3,51627,3,3
4,28758,0,0
5,31048,1,1
6,1513,3,3
7,1218,3,3
8,8322,2,2
9,21160,3,10


In [6]:
source_check_query = """
SELECT
    auditor_report.location_id,
    auditor_report.type_of_water_source AS auditor_source,
    water_source.type_of_water_source AS survey_source,
    visits.record_id,
    auditor_report.true_water_source_score AS auditor_score,
    water_quality.subjective_quality_score AS surveyor_score
FROM
    auditor_report
JOIN
    visits ON auditor_report.location_id = visits.location_id
JOIN
    water_quality ON visits.record_id = water_quality.record_id
JOIN
    water_source ON visits.source_id = water_source.source_id
WHERE
    auditor_report.true_water_source_score != water_quality.subjective_quality_score;
"""

# Step 3: Execute the query and get the DataFrame
joined_df = execute_query(source_check_query)

# Drop duplicates if necessary
joined_df = joined_df.drop_duplicates()

# Limit to 10 rows
limited_joined_df = joined_df.head(10)

# Step 4: Drop any duplicate columns (if necessary) and clean up the column names
limited_joined_df = limited_joined_df.drop(columns=['location_id'], errors='ignore')  # Drop location_id if present
limited_joined_df.rename(columns={
    'auditor_score': 'auditor_score',  # Just to clarify
    'surveyor_score': 'surveyor_score'
}, inplace=True)

# Step 5: Apply custom table styles and display the styled DataFrame
styled_joined_df = (
    limited_joined_df.style
    .applymap(highlight_cells)  # Apply custom function to highlight cells
    .set_table_styles(color_header())  # Apply custom header style
)

# Display the DataFrame with styles
print("Location ID, Record ID, Auditor Score, Surveyor Score (for unique visits):")
display(styled_joined_df)


Location ID, Record ID, Auditor Score, Surveyor Score (for unique visits):


,auditor_source,survey_source,record_id,auditor_score,surveyor_score
0,well,well,21160,3,10
1,shared_tap,shared_tap,7938,3,10
2,shared_tap,shared_tap,8076,3,10
3,shared_tap,shared_tap,8108,3,10
4,shared_tap,shared_tap,8168,3,10
5,shared_tap,shared_tap,8261,3,10
6,shared_tap,shared_tap,8277,3,10
7,shared_tap,shared_tap,8360,3,10
8,shared_tap,shared_tap,8386,3,10
9,tap_in_home_broken,tap_in_home_broken,43140,9,10


In [7]:
# Updated query to include employee information
source_check_query = """
SELECT
    auditor_report.location_id,
    auditor_report.type_of_water_source AS auditor_source,
    visits.record_id,
    auditor_report.true_water_source_score AS auditor_score,
    water_quality.subjective_quality_score AS surveyor_score,
    visits.assigned_employee_id
FROM
    auditor_report
JOIN
    visits ON auditor_report.location_id = visits.location_id
JOIN
    water_quality ON visits.record_id = water_quality.record_id
WHERE
    auditor_report.true_water_source_score != water_quality.subjective_quality_score LIMIT 10;
"""

# Step 3: Execute the query and get the DataFrame
joined_df = execute_query(source_check_query)

# Drop duplicates if necessary
joined_df = joined_df.drop_duplicates()

# Step 4: Clean up the column names (if necessary)
joined_df.rename(columns={
    'auditor_score': 'auditor_score',
    'surveyor_score': 'surveyor_score'
}, inplace=True)

# Step 5: Apply custom table styles and display the styled DataFrame
styled_joined_df = (
    joined_df.style
    .applymap(highlight_cells)  # Apply custom function to highlight cells
    .set_table_styles(color_header())  # Apply custom header style
)

# Display the DataFrame with styles
print("Location ID, Record ID, Auditor Score, Surveyor Score, Assigned Employee ID (for unique visits):")
display(styled_joined_df)


Location ID, Record ID, Auditor Score, Surveyor Score, Assigned Employee ID (for unique visits):


,location_id,auditor_source,record_id,auditor_score,surveyor_score,assigned_employee_id
0,AkRu05215,well,21160,3,10,34
1,KiRu29290,shared_tap,7938,3,10,1
2,KiRu29290,shared_tap,8076,3,10,1
3,KiRu29290,shared_tap,8108,3,10,1
4,KiRu29290,shared_tap,8168,3,10,1
5,KiRu29290,shared_tap,8261,3,10,1
6,KiRu29290,shared_tap,8277,3,10,1
7,KiRu29290,shared_tap,8360,3,10,1
8,KiRu29290,shared_tap,8386,3,10,1
9,KiHa22748,tap_in_home_broken,43140,9,10,1


In [8]:
# Updated query to include employee names
source_check_query = """
SELECT
    auditor_report.location_id,
    auditor_report.type_of_water_source AS auditor_source,
    visits.record_id,
    auditor_report.true_water_source_score AS auditor_score,
    water_quality.subjective_quality_score AS surveyor_score,
    employee.employee_name  -- Fetch employee names
FROM
    auditor_report
JOIN
    visits ON auditor_report.location_id = visits.location_id
JOIN
    water_quality ON visits.record_id = water_quality.record_id
JOIN
    employee ON visits.assigned_employee_id = employee.assigned_employee_id
WHERE
    auditor_report.true_water_source_score != water_quality.subjective_quality_score;
"""

# Step 3: Execute the query and get the DataFrame
joined_df = execute_query(source_check_query)

# Drop duplicates if necessary
joined_df = joined_df.drop_duplicates()

# Step 4: Clean up the column names (if necessary)
joined_df.rename(columns={
    'auditor_score': 'auditor_score',
    'surveyor_score': 'surveyor_score'
}, inplace=True)

# Step 5: Apply custom table styles and display the styled DataFrame
styled_joined_df = (
    joined_df.style
    .applymap(highlight_cells)  # Apply custom function to highlight cells
    .set_table_styles(color_header())  # Apply custom header style
)

# Display the DataFrame with styles
print("Location ID, Record ID, Auditor Score, Surveyor Score, Employee Name (for unique visits):")
display(styled_joined_df)


Location ID, Record ID, Auditor Score, Surveyor Score, Employee Name (for unique visits):


,location_id,auditor_source,record_id,auditor_score,surveyor_score,employee_name
0,AkRu05215,well,21160,3,10,Rudo Imani
1,KiRu29290,shared_tap,7938,3,10,Bello Azibo
2,KiRu29290,shared_tap,8076,3,10,Bello Azibo
3,KiRu29290,shared_tap,8108,3,10,Bello Azibo
4,KiRu29290,shared_tap,8168,3,10,Bello Azibo
5,KiRu29290,shared_tap,8261,3,10,Bello Azibo
6,KiRu29290,shared_tap,8277,3,10,Bello Azibo
7,KiRu29290,shared_tap,8360,3,10,Bello Azibo
8,KiRu29290,shared_tap,8386,3,10,Bello Azibo
9,KiHa22748,tap_in_home_broken,43140,9,10,Bello Azibo


In [9]:
# Query to count mistakes by employee
mistakes_count_query = """
SELECT 
    employee.employee_name,  -- Fetch employee names
    COUNT(*) AS mistakes_count
FROM
    auditor_report
JOIN
    visits ON auditor_report.location_id = visits.location_id
JOIN
    water_quality ON visits.record_id = water_quality.record_id
JOIN
    employee ON visits.assigned_employee_id = employee.assigned_employee_id
WHERE
    auditor_report.true_water_source_score != water_quality.subjective_quality_score
GROUP BY 
    employee.employee_name
ORDER BY 
    mistakes_count DESC;  -- Order by the number of mistakes in descending order
"""

# Step 5: Execute the mistakes count query and get the DataFrame
mistakes_count_df = execute_query(mistakes_count_query)

# Drop duplicates if necessary
mistakes_count_df = mistakes_count_df.drop_duplicates()

# Step 6: Clean up the column names (if necessary)
mistakes_count_df.rename(columns={
    'mistakes_count': 'mistakes_count',  # Just to clarify
}, inplace=True)

# Step 7: Apply custom table styles and display the styled DataFrame
styled_mistakes_count_df = (
    mistakes_count_df.style
    .applymap(highlight_cells)  # Apply custom function to highlight cells
    .set_table_styles(color_header())  # Apply custom header style
)

# Display the mistakes count DataFrame
print("Mistakes Count by Employee:")
display(styled_mistakes_count_df)


Mistakes Count by Employee:


,employee_name,mistakes_count
0,Bello Azibo,54
1,Malachi Mavuso,35
2,Zuriel Matembo,31
3,Rudo Imani,19
4,Lalitha Kaburi,14
5,Enitan Zuri,11
6,Xola Uzuri,8
7,Farai Nia,4
8,Yewande Ebele,3
9,Jengo Tumaini,3


In [10]:
# Execute the query to create the view (this will not return any results)
create_incorrect_records_query = """
CREATE VIEW Incorrect_records AS
SELECT
    auditor_report.location_id,
    visits.record_id,
    employee.employee_name,  -- Fetch employee names
    auditor_report.true_water_source_score AS auditor_score,
    wq.subjective_quality_score AS surveyor_score,
    auditor_report.statements AS statements
FROM
    auditor_report
JOIN
    visits ON auditor_report.location_id = visits.location_id
JOIN
    water_quality AS wq ON visits.record_id = wq.record_id
JOIN
    employee ON employee.assigned_employee_id = visits.assigned_employee_id
WHERE
    visits.visit_count = 1
    AND auditor_report.true_water_source_score != wq.subjective_quality_score;
"""
execute_query(create_incorrect_records_query)
print("View 'Incorrect_records' created.")
# Step 6: Query data from the created view
fetch_incorrect_records_query = """
SELECT * FROM Incorrect_records;
"""

# Execute the query to fetch the data from the view
incorrect_records_df = execute_query(fetch_incorrect_records_query)

# Step 7: Apply custom table styles and display the styled DataFrame
styled_incorrect_records_df = (
    incorrect_records_df.style
    .applymap(highlight_cells)  # Apply custom function to highlight cells
    .set_table_styles(color_header())  # Apply custom header style
)

print('Incorrect Records:')
display(styled_incorrect_records_df)

Query executed: 
CREATE VIEW Incorrect_records AS
SELECT
    auditor_report.location_id,
    visits.record_id,
    employee.employee_name,  -- Fetch employee names
    auditor_report.true_water_source_score AS auditor_score,
    wq.subjective_quality_score AS surveyor_score,
    auditor_report.statements AS statements
FROM
    auditor_report
JOIN
    visits ON auditor_report.location_id = visits.location_id
JOIN
    water_quality AS wq ON visits.record_id = wq.record_id
JOIN
    employee ON employee.assigned_employee_id = visits.assigned_employee_id
WHERE
    visits.visit_count = 1
    AND auditor_report.true_water_source_score != wq.subjective_quality_score;

View 'Incorrect_records' created.
Incorrect Records:


,location_id,record_id,employee_name,auditor_score,surveyor_score,statements
0,AkRu05215,21160,Rudo Imani,3,10,"Villagers admired the official's visit for its respectful interactions, hard work, and genuine concern."
1,KiRu29290,7938,Bello Azibo,3,10,"A young artist sketches the faces in the queue, capturing the weariness of daily hours spent waiting for water."
2,KiHa22748,43140,Bello Azibo,9,10,"A young girl's hopeful eyes are clouded by mistrust, her innocence tarnished by the corrupt system."
3,SoRu37841,18495,Rudo Imani,6,10,"The official's respectful and diligent presence was met with heartfelt appreciation, creating a sense of closeness with the villagers."
4,KiRu27884,33931,Bello Azibo,1,10,"A traditional healer's empathy turns to bitterness, knowing that corrupt practices harm her community."
5,KiZu31170,17950,Zuriel Matembo,9,10,"A community leader stood with his people, expressing concern for the water quality and the time lost in queues."","
6,KiZu31370,36864,Yewande Ebele,3,10,"With a keen understanding of urban challenges, the official's visit left a lasting impression of respect and commitment."
7,AkRu06495,45924,Bello Azibo,2,10,"A healthcare worker in the queue expressed fears about water-borne diseases, her face etched with worry."","
8,HaRu17528,30524,Jengo Tumaini,1,10,"With humility and diligence, the official formed bonds with the villagers that felt like genuine family connections."
9,SoRu38331,13192,Zuriel Matembo,3,10,"An unsettling atmosphere surrounded the official, as villagers shared their experiences of arrogance and lack of dedication. The mention of cash exchanges only intensified their doubts."


In [11]:
# Drop duplicates if necessary
mistakes_count_df = mistakes_count_df.drop_duplicates()

# Step 6: Clean up the column names (if necessary)
mistakes_count_df.rename(columns={
    'number_of_mistakes': 'mistakes_count',  # Just to clarify
}, inplace=True)

# Step 7: Apply custom table styles and display the styled DataFrame
styled_mistakes_count_df = (
    mistakes_count_df.style
    .applymap(highlight_cells)  # Apply custom function to highlight cells
    .set_table_styles(color_header())  # Apply custom header style
)

# Display the mistakes count DataFrame
print("Mistakes Count by Employee:")
display(styled_mistakes_count_df)

Mistakes Count by Employee:


,employee_name,mistakes_count
0,Bello Azibo,54
1,Malachi Mavuso,35
2,Zuriel Matembo,31
3,Rudo Imani,19
4,Lalitha Kaburi,14
5,Enitan Zuri,11
6,Xola Uzuri,8
7,Farai Nia,4
8,Yewande Ebele,3
9,Jengo Tumaini,3


In [12]:
# Step 2: SQL query to calculate the average number of mistakes in error_count
avg_error_count_query = """
SELECT 
    AVG(number_of_mistakes) AS avg_error_count_per_empl
FROM 
    error_count;
"""

# Execute the query to get the average number of mistakes
avg_error_count_df = execute_query(avg_error_count_query)

# Display the average number of mistakes
print("Average Number of Mistakes per Employee:")
display(avg_error_count_df)

ProgrammingError: 1146 (42S02): Table 'md_water_services.error_count' doesn't exist

In [ ]:
# Step 3: SQL query to find employees with above-average mistakes
suspect_list_query = """
SELECT 
    employee_name,
    number_of_mistakes
FROM 
    error_count
WHERE 
    number_of_mistakes > (
        SELECT 
            AVG(number_of_mistakes) 
        FROM 
            error_count
    );
"""

# Execute the query and get the suspect list DataFrame
suspect_list_df = execute_query(suspect_list_query)

# Apply custom styling to the DataFrame
styled_suspect_list_df = (
    suspect_list_df.style
    .applymap(highlight_cells)  # Apply custom function to highlight cells
    .set_table_styles(color_header())  # Apply custom header style
)

# Display the styled DataFrame
print("Employees with Above-Average Mistakes:")
display(styled_suspect_list_df)


In [ ]:
config = {
    'user': 'root',
    'password': 'Bestest1234',
    'host': 'localhost',
    'database': 'md_water_services',
    'raise_on_warnings': True
}
def execute_query(query):
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    cursor.execute(query)
    if cursor.description is not None:  # Check if the query returns a result
        df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
    else:
        conn.commit()  # Commit the changes if it's an UPDATE query
        cursor.close()
        conn.close()
    return df

In [ ]:
# Step 1: SQL query to create the CTE for suspect_list
suspect_list_query = """
WITH suspect_list AS (
    SELECT 
        employee_name
    FROM 
        error_count
    WHERE 
        number_of_mistakes > (
            SELECT 
                AVG(number_of_mistakes) 
            FROM 
                error_count
        )
)
SELECT 
    *
FROM 
    Incorrect_records
WHERE 
    employee_name IN (SELECT employee_name FROM suspect_list);
"""

# Execute the query to get records associated with suspect employees
suspect_records_df = execute_query(suspect_list_query)

# Step 2: Apply custom styling to the DataFrame
styled_suspect_records_df = (
    suspect_records_df.style
    .applymap(highlight_cells)  # Apply custom function to highlight cells
    .set_table_styles(color_header())  # Apply custom header style
)

# Step 3: Display the styled DataFrame
print("Records Associated with Suspect Employees:")
display(styled_suspect_records_df)